In [1]:
from residual_learning.utils import BaseForecaster
import pandas as pd
import matplotlib.pyplot as plt
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
targets = pd.read_csv("targets.csv.gz")

In [2]:
targets

,datetime,site_id,chla,oxygen,temperature,air_tmp
0,2016-03-05,ARIK,NaN,NaN,NaN,NaN
1,2016-03-06,ARIK,NaN,NaN,NaN,NaN
2,2016-03-07,ARIK,NaN,NaN,NaN,NaN
3,2016-03-08,ARIK,NaN,NaN,NaN,NaN
4,2016-03-09,ARIK,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
60348,2023-07-07,WLOU,NaN,8.878493,5.738894,NaN
60349,2023-07-08,WLOU,NaN,8.791903,6.059050,NaN
60350,2023-07-09,WLOU,NaN,8.819632,5.998123,NaN
60351,2023-07-10,WLOU,NaN,8.792500,6.141779,NaN


In [3]:
block_rnn_forecaster = BaseForecaster(model="BlockRNN",
                                      input_csv_name="targets.csv.gz",
                                      target_variable_column_name="oxygen",
                                      datetime_column_name="datetime",
                                      covariates_names=["air_tmp", "chla", "temperature"],
                                      output_csv_name="trash.csv",
                                      validation_split_date="2023-02-20",
                                      model_hyperparameters={'input_chunk_length': 180, 
                                                             'hidden_dim': 64, 
                                                             'model': 'GRU', 
                                                             'n_rnn_layers': 3},
                                     forecast_horizon=30,
                                     site_id="ARIK")

You are transforming a stochastic TimeSeries (i.e., contains several samples). The resulting DataFrame is a 2D object with all samples on the columns. If this is not the expected behavior consider calling a function adapted to stochastic TimeSeries like quantile_df().
You are transforming a stochastic TimeSeries (i.e., contains several samples). The resulting DataFrame is a 2D object with all samples on the columns. If this is not the expected behavior consider calling a function adapted to stochastic TimeSeries like quantile_df().
You are transforming a stochastic TimeSeries (i.e., contains several samples). The resulting DataFrame is a 2D object with all samples on the columns. If this is not the expected behavior consider calling a function adapted to stochastic TimeSeries like quantile_df().
You are transforming a stochastic TimeSeries (i.e., contains several samples). The resulting DataFrame is a 2D object with all samples on the columns. If this is not the expected behavior consi

Failed to make a GP fit on air_tmp
Failed to make a GP fit on chla


In [ ]:
block_rnn_forecaster.make_forecasts()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


{'input_chunk_length': 180, 'hidden_dim': 64, 'model': 'GRU', 'n_rnn_layers': 3}


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


In [ ]:
df = pd.read_csv("trash.csv")
times = pd.to_datetime(df["datetime"])
times = pd.DatetimeIndex(times)
values = df.loc[:, df.columns!="datetime"].to_numpy().reshape((block_rnn_forecaster.forecast_horizon, 1, -1))
direct_predictions_ts = TimeSeries.from_times_and_values(times, 
                                                         values, 
                                                         fill_missing_dates=True, freq="D")

plt.clf()
direct_predictions_ts.plot(color="salmon", label="Forecast")
block_rnn_forecaster.validation_set[:block_rnn_forecaster.forecast_horizon].plot(color="black", label="Truth")
plt.show()